In [32]:
from firedanger import firedanger
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from cartopy import crs as ccrs
from cartopy import feature as cfeature

In [33]:
# %history -g -f history.txt

In [34]:
del fire

In [68]:
workdir = "/work/bb1153/m300363/fireweather_data/California/"
# infile = workdir + 'inputvars_ICON_ngc4008_California_025deg_2020-2049.nc'

# infile = workdir + "inputvars_ICON_ngc4008_California_025deg_2021.nc"
# outfile = workdir + "fwi_test_ICON.nc"

infile = workdir + "inputvars_IFS_9-FESOM_5-production_California_025deg_2023.nc"
outfile = workdir + "fwi_test_IFS_9-FESOM_5_2023.nc"

# infile = workdir + "allvars_2021.nc"
# outfile = workdir + "test.nc"

# infile = workdir + "out.nc"
# outfile = workdir + 'fwi_ICON_ngc4008_California_025deg_2020-2049.nc'

# infile = workdir + "testfiles_noemi/allvars_merged.nc"
# outfile =  workdir + "testfiles_noemi/fwi.nc"

fire = firedanger() 
fire.read_nc(infile)

In [65]:
help(fire.calc_canadian_fwi)

Help on method calc_canadian_fwi in module firedanger.firedanger:

calc_canadian_fwi(temp, precip, hum, wind) method of firedanger.firedanger.firedanger instance
    Calculate the Canadian Fire Weather Index 
    
    Parameters
    ----------
    t: array
        Noon temperature [C].
    p : array
        rainfall amount in open over previous 24 hours, at noon [mm].
    w : array
        Noon wind speed [m/s].
    h : array
        Noon relative humidity [%].
    
    Returns
    -------
    ds: xarray dataset
        Components of the Canadian Fire Weather Index: ffmc, dmc, dc, isi, bui, fwi [dimensionless]
    
    To DO:
    - latitude array as input --> does not yet work in indices._day_length
      currently, lat = 47 (Switzerland)
    - initial values of ffmc, dmc and dc
    - cannot handle dask array yet: current solution is to load dataset



In [ ]:
# fire.tas

# fire

In [69]:
fire.calc_canadian_fwi("tas","pr","hurs","wind_speed")

INFO: Set up dimensions...
INFO: 
 time: 'time'
 longitude: 'lon'
 latitude: 'lat'

/home/m/m300363/.conda/envs/firedanger_dev/lib/python3.8/site-packages/firedanger/indices.py:1148: RuntimeWarning: invalid value encountered in true_divide
  (0.8 * dc * dmc) / (dmc + 0.4 * dc),
/home/m/m300363/.conda/envs/firedanger_dev/lib/python3.8/site-packages/firedanger/indices.py:1150: RuntimeWarning: invalid value encountered in true_divide
  dmc - (1.0 - 0.8 * dc / (dmc + 0.4 * dc)) * (0.92 + (0.0114 * dmc) ** 1.7),
INFO: Calculating Canadian Fire Weather Index (ffmc, dmc, dc, isi, bui, fwi)... DONE


In [70]:
dsout = xr.Dataset({
    "fwi": (("time", "lat", "lon"), fire.fwi , {"units": "dimensionless"}),
    "bui": (("time", "lat", "lon"), fire.bui , {"units": "dimensionless"}),
    "isi": (("time", "lat", "lon"), fire.isi , {"units": "dimensionless"}),
    "dmc": (("time", "lat", "lon"), fire.dmc , {"units": "dimensionless"}),
    "dc": (("time", "lat", "lon"), fire.dc , {"units": "dimensionless"}),
    "ffmc": (("time", "lat", "lon"), fire.ffmc , {"units": "dimensionless"})},
    coords={
        "time": fire.fwi.time,
        "lat": ("lat", fire.lat, {"units": "degree_north"}),
        "lon": ("lon", fire.lon, {"units": "degree_east"}),
        },)

dsout.to_netcdf(outfile)